In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Leitura dos dados do dataframe
df = pd.read_csv('datasets/(processado-final)textos_tuitesPt_2020.csv.gz', names=['texto'])

# Elimina um valor flutuante que aparece no dataframe (por razões misteriosas)
# o algoritmo não aceita o valor flutuante, que precisa ser filtrado
df = df[df['texto'].apply(lambda x: isinstance(x, str))]
df['texto'].apply(type).value_counts()

df = df.head(500000)

df

,texto
0,coronavirus aceitar braco abrir
1,achar eleitor bolsonaro medo coronavirus febre...
2,fome coronavirus entrar fila
3,trancar replies twetr sala coronavirus
4,caso coronavirus confirmar Brasil mundo querer...
...,...
499996,preciso acalmar puertorico boardgame jogosdeta...
499997,assustar aparentemente virus covid vez agir vi...
499998,bem bom perder milhar vida covid19 fiquemcasa
499999,testir saber infectar


In [3]:
# Define o número de tópicos desejados
n_components = 10

# Cria o objeto TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=None)

# Cria a matriz de termo-documento
X = vectorizer.fit_transform(df['texto'])

In [4]:
# Executa o NMF
nmf = NMF(n_components=n_components, init='nndsvd', random_state=1)
W = nmf.fit_transform(X)
H = nmf.components_

In [5]:
# Obtém os tópicos para o primeiro documento
doc = df['texto'][0]
tfidf_doc = vectorizer.transform([doc])
topic_scores = tfidf_doc.dot(H.T)
topic_indices = topic_scores.argsort()[::-1][:n_components]

In [7]:
# Obtém as palavras correspondentes a cada coluna da matriz de frequência de termos
feature_names = vectorizer.get_feature_names()

# Imprime as palavras correspondentes a cada tópico
print("Palavras e pesos dos tópicos:")
for i, topic in enumerate(H):
    top_words_indices = topic.argsort()[:-11:-1]
    topic_words = [feature_names[j] for j in top_words_indices]
    topic_weights = topic[top_words_indices]
    print("Tópico {}: {}".format(i, [(word, weight) for word, weight in zip(topic_words, topic_weights)]))

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'

In [ ]:
# Calcula a perplexidade do modelo
term_frequency = X.sum(axis=1)
term_probability = np.array(term_frequency / term_frequency.sum())
doc_topic_probability = W / W.sum(axis=1).reshape(-1, 1)
topic_word_probability = H / H.sum(axis=1).reshape(-1, 1)
perplexity = np.exp(-np.sum(term_probability * np.log(doc_topic_probability @ topic_word_probability)))
print("Perplexidade do modelo: {:.2f}".format(perplexity))